In [6]:
# ! pip install iotbr==0.1.0
import numpy as np
import pandas as pd
from iotbr import tru as tru
#from iotbr import tru_pb as tru_pb
from iotbr import io_system as ios

In [7]:
# 1) simular choque (sistema aberto)
#considera apenas o efeito direto
#existem três tipos de efeitos: direto, indireto e induzido
sys = ios.system('2019','68','t')
mL = sys.mL
#suponha um aumento de10% nas exportações:
vDY = sys.mY['X_bens_serv'] * 0.1
vDX = np.dot(mL,vDY )


#variação da demanda total (dataframe)
vX = tru.read_var('2019','68','VA_table','t')['Valor da produção'].values
DX_df = pd.DataFrame({'setor': vDY.index.tolist(),
        'DX': vDX / vX})

DX_df['DX'] = DX_df['DX'].map('{:.2%}'.format)
DX_df

,setor,DX
0,"0191\nAgricultura, inclusive o apoio à agricul...",4.43%
1,"0192\nPecuária, inclusive o apoio à pecuária",1.70%
2,0280\nProdução florestal; pesca e aquicultura,1.99%
3,0580\nExtração de carvão mineral e de minerais...,2.54%
4,"0680\nExtração de petróleo e gás, inclusive as...",4.83%
...,...,...
63,8691\nSaúde pública,0.00%
64,8692\nSaúde privada,0.02%
65,"9080\nAtividades artísticas, criativas e de es...",0.82%
66,9480\nOrganizações associativas e outros servi...,0.23%


In [8]:
# 2) simular choque (sistema fechado)
#Considera o efeito direto e o efeito induzido
import numpy as np
from iotbr import io_system as ios
sys = ios.system('2019','68','t')
mL_h = sys.mL_h

#suponha um aumento de10% nas exportações:
#vDY = sys.mY['X_bens_serv'] * 0.1
vDY_ = vDY.copy()

#adicione o setor 'household' np vetor vDY
# considere que não esportamos 'household'
import pandas as pd
#vDY_.loc[len(vDY_)] = 0
nDY_h = pd.Series([0], index=['household'])
vDY_ = pd.concat([vDY_, nDY_h])


#camcular variação na demanda total causada pela variação na demanda final:
vDX_ = np.dot(mL_h,vDY_ )
vDX_

#variação da demanda total (dataframe)
#ignorar setor de household (causaria divisão por zero pois nX_h = 0)
vDX_tru = vDX_[0:int('68')] #truncado
vX = tru.read_var('2019','68','VA_table','t')['Valor da produção'].values
DX_df = pd.DataFrame({'setor': vDY.index.tolist(),
        'DX_': vDX_tru / vX})

DX_df['DX_'] = DX_df['DX_'].map('{:.2%}'.format)
DX_df

,setor,DX_
0,"0191\nAgricultura, inclusive o apoio à agricul...",5.31%
1,"0192\nPecuária, inclusive o apoio à pecuária",2.90%
2,0280\nProdução florestal; pesca e aquicultura,3.20%
3,0580\nExtração de carvão mineral e de minerais...,3.04%
4,"0680\nExtração de petróleo e gás, inclusive as...",5.44%
...,...,...
63,8691\nSaúde pública,0.02%
64,8692\nSaúde privada,1.37%
65,"9080\nAtividades artísticas, criativas e de es...",2.00%
66,9480\nOrganizações associativas e outros servi...,1.34%


In [9]:
# 3) comparar sistema aberto e sistema fechado

#aumento de 10% no consumo das fammilias
sys = ios.system('2019','68','t')
vDY = sys.mY['C_f'] * 0.1

#variacao da demanda final (modelo aberto)
vDX = np.dot(sys.mL,vDY )

#variacao da demanda final (modelo fechado)
import pandas as pd
nDY_h = pd.Series([0], index=['household'])
vDY_ = pd.concat([vDY, nDY_h])
vDX_ = np.dot(sys.mL_h,vDY_ )

#variação da demanda total (dataframe)

#ignorar setor de household (causaria divisão por zero pois nX_h = 0)
vDX_tru = vDX_[0:int('68')] #truncado
vX = tru.read_var('2019','68','VA_table','t')['Valor da produção'].values
DX_df = pd.DataFrame({'setor': vDY.index.tolist(),
        'DX modelo aberto': vDX / vX,
        'DX modelo fechado': vDX_tru / vX})

DX_df['DX modelo aberto'] = DX_df['DX modelo aberto'].map('{:.2%}'.format)
DX_df['DX modelo fechado'] = DX_df['DX modelo fechado'].map('{:.2%}'.format)
DX_df

,setor,DX modelo aberto,DX modelo fechado
0,"0191\nAgricultura, inclusive o apoio à agricul...",5.22%,9.22%
1,"0192\nPecuária, inclusive o apoio à pecuária",7.05%,12.46%
2,0280\nProdução florestal; pesca e aquicultura,7.12%,12.58%
3,0580\nExtração de carvão mineral e de minerais...,2.97%,5.26%
4,"0680\nExtração de petróleo e gás, inclusive as...",3.59%,6.35%
...,...,...,...
63,8691\nSaúde pública,0.11%,0.20%
64,8692\nSaúde privada,8.03%,14.18%
65,"9080\nAtividades artísticas, criativas e de es...",6.98%,12.32%
66,9480\nOrganizações associativas e outros servi...,6.54%,11.56%


In [10]:
# 4) outros choques
#aumento de 10% no consumo das fammilias
sys = ios.system('2019','68','t')
vDY = sys.mY['C_f'] * 0.1

#variacao da demanda final (modelo aberto)
vDX = np.dot(sys.mL,vDY ).reshape(-1, 1)

#variacao da demanda final (modelo fechado)
nDY_h = pd.Series([0], index=['household'])
vDY_ = pd.concat([vDY, nDY_h])
vDX_ = np.dot(sys.mL_h,vDY_ ).reshape(-1, 1)

#ignorar setor de household (não entendi porque isso é feito)
vDX_tru = vDX_[0:int('68')].reshape(-1, 1) #truncado

#Variacao do VA
vX = tru.read_var('2019','68','VA_table','t')['Valor da produção'].values.reshape(-1, 1)
mVA = tru.read_var('2019','68','VA_table','t')
VA_df = pd.DataFrame()
for i in mVA.columns:
  vVA = tru.read_var('2019','68','VA_table','t')[i].values.reshape(-1, 1)
  VA_df_ = pd.DataFrame({
      "Variable": [i],
      "Modelo Abero (Variação em BRL)": [np.sum((vDX / vX) * vVA)],
      "Modelo Abero (Variação em %)": [np.sum((vDX / vX) * vVA) / np.sum(vVA)],
      "Modelo fechado (Variação em BRL)": [np.sum((vDX_tru / vX) * vVA)],
      "Modelo fechado (Variação em %)": [np.sum((vDX_tru / vX) * vVA / np.sum(vVA))],})
  VA_df = pd.concat([VA_df, VA_df_], axis=0)



#Variacao dos Impostos
T_int_df = pd.DataFrame()
for i in ['I_imp','ICMS','IPI','OI_liq_Sub']:
  vT_int = sys.mT_int[i].values.reshape(-1, 1)
  T_int_df_ = pd.DataFrame({
      "Variable": [i],
      "Modelo Abero (Variação em BRL)": [np.sum((vDX / vX) * vT_int)],
      "Modelo Abero (Variação em %)": [np.sum((vDX / vX) * vT_int) / np.sum(vT_int)],
      "Modelo fechado (Variação em BRL)": [np.sum((vDX_tru / vX) * vT_int)],
      "Modelo fechado (Variação em %)": [np.sum((vDX_tru / vX) * vT_int / np.sum(vT_int))],})
  T_int_df = pd.concat([T_int_df, T_int_df_], axis=0)
T_int_df

df = pd.concat([VA_df, T_int_df], axis=0)
df.set_index('Variable', inplace=True)
df['Modelo Abero (Variação em BRL)'] = df['Modelo Abero (Variação em BRL)'].map('{:.2f}'.format)
df['Modelo Abero (Variação em %)'] = df['Modelo Abero (Variação em %)'].map('{:.2%}'.format)
df['Modelo fechado (Variação em BRL)'] = df['Modelo fechado (Variação em BRL)'].map('{:.2f}'.format)
df['Modelo fechado (Variação em %)'] = df['Modelo fechado (Variação em %)'].map('{:.2%}'.format)
df

,Modelo Abero (Variação em BRL),Modelo Abero (Variação em %),Modelo fechado (Variação em BRL),Modelo fechado (Variação em %)
Variable,,,,
Valor adicionado bruto ( PIB ),340587.33,5.36%,601735.84,9.47%
Remunerações,139644.72,4.34%,246718.61,7.67%
Salários,113042.23,4.45%,199718.42,7.86%
Contribuições sociais efetivas,26281.68,4.59%,46433.40,8.11%
Previdência oficial /FGTS,24947.15,4.54%,44075.61,8.01%
Previdência privada,1334.53,5.85%,2357.79,10.33%
Contribuições sociais imputadas,320.81,0.31%,566.79,0.54%
Excedente operacional bruto e rendimento misto bruto,195291.21,6.41%,345032.55,11.32%
Rendimento misto bruto,36327.38,5.88%,64181.74,10.39%


In [11]:
# 5) Calculando Multiplicadores de Impacto
#Multiplicador de produção

#Sistema Leontief
sys = ios.system('2019','68','t')

#ignorar setor de household (evita divisão por zero)
mL_h_tru = sys.mL_h.iloc[:,0:int('68')] #truncado

#Calculo do multiplicador de impacto
vMultTotalProducao = np.sum(mL_h_tru, axis=0)
vMultSimplesProducao = np.sum(sys.mL, axis=0)
vEfeitoInduzido = vMultTotalProducao - vMultSimplesProducao

mI = np.eye(int('68'))
mA = mI - np.linalg.inv(sys.mL)
vEfeitoDireto = np.sum(mA, axis=0)
vEfeitoIndireto =  vMultSimplesProducao - vEfeitoDireto

mult_PT_df = pd.DataFrame({'Total': vMultTotalProducao,
                           'Direto+Indireto': vMultSimplesProducao,
                           'Induzido': vEfeitoInduzido,
                           'Direto': vEfeitoDireto,
                           'Indireto': vEfeitoIndireto})
mult_PT_df

,Total,Direto+Indireto,Induzido,Direto,Indireto
"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita",2.822408,1.757990,1.064418,0.394722,1.363268
"0192\nPecuária, inclusive o apoio à pecuária",3.414784,1.880044,1.534740,0.447622,1.432422
0280\nProdução florestal; pesca e aquicultura,2.200214,1.432581,0.767633,0.240501,1.192080
0580\nExtração de carvão mineral e de minerais não-metálicos,3.869448,1.889168,1.980281,0.474517,1.414651
"0680\nExtração de petróleo e gás, inclusive as atividades de apoio",2.830931,1.682879,1.148052,0.380991,1.301888
...,...,...,...,...,...
8691\nSaúde pública,5.567329,1.495661,4.071668,0.289014,1.206647
8692\nSaúde privada,4.080499,1.601799,2.478700,0.365060,1.236740
"9080\nAtividades artísticas, criativas e de espetáculos",4.027610,1.600944,2.426666,0.379303,1.221641
9480\nOrganizações associativas e outros serviços pessoais,3.915525,1.676306,2.239218,0.394544,1.281763


In [12]:
# 6) Multiplicador de Emprego
# Versão João Maria
vOcupacoes = tru.read_var('2019','68','VA_table','t')['Fator trabalho (ocupações)'].values.reshape(-1, 1)
vVBP = tru.read_var('2019','68','VA_table','t')['Valor da produção'].values.reshape(-1, 1)
vRequisitosEmprego = vOcupacoes / vVBP
mRequisitosEmpregoDiagonal = np.diagflat(vRequisitosEmprego)
mGeradorEmprego = np.dot(mRequisitosEmpregoDiagonal, sys.mL)
vMultSimplesEmprego = np.sum(mGeradorEmprego, axis=0)
vMultiplicadorEmpregoI =  vMultSimplesEmprego / vRequisitosEmprego.reshape(-1,)

mGeradorEmpregoModeloFechado = np.dot(mRequisitosEmpregoDiagonal, sys.mL_h.iloc[0:int('68'),0:int('68')])
vMultTotalEmprego = np.sum(mGeradorEmpregoModeloFechado, axis=0)
vMultiplicadorEmpregoII =  vMultTotalEmprego / vRequisitosEmprego.reshape(-1,)

vEfeitoInduzidoEmprego = vMultTotalEmprego - vMultSimplesEmprego
vEfeitoDiretoEmprego  = np.sum(np.dot(mRequisitosEmpregoDiagonal, mA), axis=0)
vEfeitoIndiretoEmprego = vMultSimplesEmprego - vEfeitoDiretoEmprego
#print("setores                                               Total  Direto+  ")
#print("                                                             Indireto Induzido Direto Indireto ")
mMultiplicadoresEmprego = np.vstack((vMultTotalEmprego, vMultSimplesEmprego,vEfeitoInduzidoEmprego,vEfeitoDiretoEmprego, vEfeitoIndiretoEmprego  )).T
#for i in range(int('68')):
#  print("{0:55} {1:6.4}  {2:6.4}  {3:6.4}  {4:6.4}  {5:6.4} ".format('setor',vMultTotalEmprego[i], vMultSimplesEmprego[i], \
#                                                                    vEfeitoInduzidoEmprego[i],vEfeitoDiretoEmprego[i], vEfeitoIndiretoEmprego[i]))

mult_W_df = pd.DataFrame({'setor': sys.mL.index.tolist(),
                          'Total': vMultTotalEmprego,
                          'Direto+Indireto': vMultSimplesEmprego,
                          'Induzido': vEfeitoInduzidoEmprego,
                          'Direto': vEfeitoDiretoEmprego,
                          'Indireto': vEfeitoIndiretoEmprego,
                          'Mult I':vMultiplicadorEmpregoI,
                          'Mult II':vMultiplicadorEmpregoII})
mult_W_df

,setor,Total,Direto+Indireto,Induzido,Direto,Indireto,Mult I,Mult II
0,"0191\nAgricultura, inclusive o apoio à agricul...",25.61764,19.227718,6.389922,1.772361,17.455357,1.226634,1.634279
1,"0192\nPecuária, inclusive o apoio à pecuária",51.86237,42.649013,9.213357,4.017104,38.631908,1.192355,1.449936
2,0280\nProdução florestal; pesca e aquicultura,30.595949,25.98769,4.608259,2.593321,23.394369,1.171686,1.379455
3,0580\nExtração de carvão mineral e de minerais...,22.042636,10.154606,11.88803,2.558579,7.596027,1.857939,4.033035
4,"0680\nExtração de petróleo e gás, inclusive as...",10.876242,3.984251,6.891992,2.063528,1.920722,19.261309,52.579691
...,...,...,...,...,...,...,...,...
63,8691\nSaúde pública,39.362533,14.919478,24.443055,3.153149,11.766328,1.446883,3.817358
64,8692\nSaúde privada,31.154524,16.274382,14.880142,3.578771,12.695611,1.487717,2.847979
65,"9080\nAtividades artísticas, criativas e de es...",43.94589,29.378115,14.567775,2.560269,26.817846,1.152614,1.724162
66,9480\nOrganizações associativas e outros servi...,44.070373,30.627887,13.442486,3.664703,26.963185,1.216948,1.751063


In [ ]:
# 7) Multiplicador de renda do trabalho
# Versão João Maria
vVA = tru.read_var('2019','68','VA_table','t')['Remunerações'].values.reshape(-1, 1)

vRequisitosRenda = vVA / vVBP
mRequisitosRendaDiagonal = np.diagflat(vRequisitosRenda)
mGeradorRenda = np.dot(mRequisitosRendaDiagonal, sys.mL)
vMultSimplesRenda = np.sum(mGeradorRenda, axis=0)
vMultiplicadorRendaI =  vMultSimplesRenda / vRequisitosRenda.reshape(-1,)

mGeradorRendaModeloFechado = np.dot(mRequisitosRendaDiagonal, sys.mL_h.iloc[0:int('68'),0:int('68')])
vMultTotalRenda = np.sum(mGeradorRendaModeloFechado, axis=0)
vMultiplicadorRendaII =  vMultTotalRenda / vRequisitosRenda.reshape(-1,)

vEfeitoInduzidoRenda = vMultTotalRenda - vMultSimplesRenda
vEfeitoDiretoRenda  = np.sum(np.dot(mRequisitosRendaDiagonal, mA), axis=0)
vEfeitoIndiretoRenda = vMultSimplesRenda - vEfeitoDiretoRenda
#print("setores                                        Total  Direto+  ")
#print("                                                     Indireto Induzido Direto Indireto  Mult I   Mult II ")
#mMultiplicadoresRenda = np.vstack((vMultTotalRenda, vMultSimplesRenda,vEfeitoInduzidoRenda,vEfeitoDiretoRenda, vEfeitoIndiretoRenda  )).T
#for i in range(int('68')):
#  print("{0:45} {1:6.4}  {2:6.4}  {3:6.4}  {4:7.4}  {5:7.4}  {6:7.4}  {7:7.4}".format(vNomeSetores[i][5:50],vMultTotalRenda[i], vMultSimplesRenda[i], \
#                                                                  vEfeitoInduzidoRenda[i],vEfeitoDiretoRenda[i], vEfeitoIndiretoRenda[i], \
#                                                                  vMultiplicadorRendaI[0, i], vMultiplicadorRendaII[0, i] ))

mult_W_df = pd.DataFrame({'setor': sys.mL.index.tolist(),
                          'Total': vMultTotalRenda,
                          'Direto+Indireto': vMultSimplesRenda,
                          'Induzido': vEfeitoInduzidoRenda,
                          'Direto': vEfeitoDiretoRenda,
                          'Mult I':vMultiplicadorRendaI,
                          'Mult II':vMultiplicadorRendaII})
mult_W_df